In [22]:
import os
import io
from IPython.display import Image, display, HTML
from PIL import Image
import base64 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
hf_api_key = os.environ['HF_API_KEY']

In [23]:
# Helper function
import gradio as gr
import requests, json

#Summarization endpoint
def get_completion(inputs, parameters=None,ENDPOINT_URL=os.environ['HF_API_SUMMARY_BASE']): 
    headers = {
      "Authorization": f"Bearer {hf_api_key}",
      "Content-Type": "application/json"
    }
    data = { "inputs": inputs }
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",
                                ENDPOINT_URL, headers=headers,
                                data=json.dumps(data)
                               )
    return json.loads(response.content.decode("utf-8"))

In [30]:
API_URL = os.environ['HF_API_NER_BASE'] 
text = ('''The tower is 324 metres (1,063 ft) tall, about the same height
        as an 81-storey building, and the tallest structure in Paris. 
        Its base is square, measuring 125 metres (410 ft) on each side. 
        During its construction, the Eiffel Tower surpassed the Washington 
        Monument to become the tallest man-made structure in the world,
        a title it held for 41 years until the Chrysler Building
        in New York City was finished in 1930. It was the first structure 
        to reach a height of 300 metres. Due to the addition of a broadcasting 
        aerial at the top of the tower in 1957, it is now taller than the 
        Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the 
        Eiffel Tower is the second tallest free-standing structure in France 
        after the Millau Viaduct.''')

get_completion(text, parameters=None, ENDPOINT_URL= API_URL)


[{'entity': 'B-LOC',
  'score': 0.9997018,
  'index': 32,
  'word': 'Paris',
  'start': 126,
  'end': 131},
 {'entity': 'B-LOC',
  'score': 0.93327284,
  'index': 55,
  'word': 'E',
  'start': 244,
  'end': 245},
 {'entity': 'I-LOC',
  'score': 0.7716576,
  'index': 56,
  'word': '##iff',
  'start': 245,
  'end': 248},
 {'entity': 'I-LOC',
  'score': 0.59214735,
  'index': 57,
  'word': '##el',
  'start': 248,
  'end': 250},
 {'entity': 'I-LOC',
  'score': 0.92810106,
  'index': 58,
  'word': 'Tower',
  'start': 251,
  'end': 256},
 {'entity': 'B-LOC',
  'score': 0.9743313,
  'index': 61,
  'word': 'Washington',
  'start': 271,
  'end': 281},
 {'entity': 'I-LOC',
  'score': 0.9720976,
  'index': 62,
  'word': 'Monument',
  'start': 291,
  'end': 299},
 {'entity': 'B-ORG',
  'score': 0.7149701,
  'index': 84,
  'word': 'Chrysler',
  'start': 402,
  'end': 410},
 {'entity': 'I-LOC',
  'score': 0.8875241,
  'index': 85,
  'word': 'Building',
  'start': 411,
  'end': 419},
 {'entity': 'B-L

In [25]:
def ner(input):
    output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
    return {"text": input, "entities": output}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    #Here we introduce a new tag, examples, easy to use examples for your application
                    examples=["My name is Shoaib and I live in Chennai", "My name is Shoaib and working with HuggingFace"])
demo.launch(share=True, server_port=int(os.environ['PORT3']))

Closing server running on port: 7862
Closing server running on port: 7862
Closing server running on port: 7862
Closing server running on port: 7861
Closing server running on port: 7860
Running on local URL:  https://0.0.0.0:7862
IMPORTANT: You are using gradio version 3.37.0, however version 4.44.1 is available, please upgrade.
--------

Could not create share link. Missing file: /usr/local/lib/python3.9/site-packages/gradio/frpc_linux_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.2
3. Move the file to this location: /usr/local/lib/python3.9/site-packages/gradio


In [26]:
def merge_tokens(tokens):
    merged_tokens = []
    for token in tokens:
        if merged_tokens and token['entity'].startswith('I-') and merged_tokens[-1]['entity'].endswith(token['entity'][2:]):
            # If current token continues the entity of the last one, merge them
            last_token = merged_tokens[-1]
            last_token['word'] += token['word'].replace('##', '')
            last_token['end'] = token['end']
            last_token['score'] = (last_token['score'] + token['score']) / 2
        else:
            # Otherwise, add the token to the list
            merged_tokens.append(token)

    return merged_tokens

def ner(input):
    output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
    merged_tokens = merge_tokens(output)
    return {"text": input, "entities": merged_tokens}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    examples=["My name is Shoaib, I'm learning through DeeplearningAI and I live in Chennai", "My name is Shoaib, I live in Chennai and working with HuggingFace"])

demo.launch(share=True, server_port=int(os.environ['PORT3']))

Closing server running on port: 7862
Closing server running on port: 7862
Closing server running on port: 7862
Closing server running on port: 7862
Closing server running on port: 7861
Closing server running on port: 7860
Running on local URL:  https://0.0.0.0:7862
IMPORTANT: You are using gradio version 3.37.0, however version 4.44.1 is available, please upgrade.
--------

Could not create share link. Missing file: /usr/local/lib/python3.9/site-packages/gradio/frpc_linux_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.2
3. Move the file to this location: /usr/local/lib/python3.9/site-packages/gradio


In [31]:
gr.close_all()

Closing server running on port: 7862
Closing server running on port: 7862
Closing server running on port: 7862
Closing server running on port: 7862
Closing server running on port: 7861
Closing server running on port: 7860
Closing server running on port: 7862
Closing server running on port: 7862
